In [1]:
import numpy as np
import pandas as pd

import data_manipulation as dm
import time
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [74]:
#Import datasets
txn = dm.transactions()
train = dm.train()

In [75]:
#Check datasets
print(train.shape)
print(txn.shape)
print(train['new_id'].nunique())
print(txn['new_id'].nunique())

train['in_txn']=train['new_id'].isin(txn['new_id'])
print(train['in_txn'].value_counts())

(992931, 2)
(16411725, 9)
992931
1079663
True    992931
Name: in_txn, dtype: int64


In [5]:
#Determine if id is in train dataset
txn['in_train']=txn['new_id'].isin(train['new_id'])
txn['odd_transactions']=txn['membership_expire_date']<txn['transaction_date']
txn['1970']=txn['membership_expire_date']=='1970-01-01'

txn['in_train'].value_counts()

True     15883148
False      528577
Name: in_train, dtype: int64

In [63]:
pd.crosstab(index=txn['odd_transactions'], columns=txn['in_train'], normalize='index')

in_train,False,True
odd_transactions,,
False,0.032114,0.967886
True,0.054375,0.945625


In [64]:
#Odd transactions are more likely to be auto renew transactions
pd.crosstab(index=txn['odd_transactions'], columns=txn['is_auto_renew'], normalize='index')

is_auto_renew,0,1
odd_transactions,,
False,0.089980,0.910020
True,0.004789,0.995211


In [65]:
#Odd transactions are more likely to be cancelled transactions
pd.crosstab(index=txn['odd_transactions'], columns=txn['is_cancel'], normalize='index')

is_cancel,0,1
odd_transactions,,
False,0.986408,0.013592
True,0.047459,0.952541


In [9]:
#397 of the odd transactions are caused by '1970'
pd.crosstab(index=txn['odd_transactions'], columns=txn['1970'], normalize='index')
pd.crosstab(index=txn['odd_transactions'], columns=txn['1970'])

1970,False,True
odd_transactions,,
False,16342612,0
True,68716,397


In [10]:
#226 bad transactions are in the training set
pd.crosstab(index=txn['in_train'], columns=txn['1970'])

1970,False,True
in_train,,
False,528406,171
True,15882922,226


In [11]:
#0.4% of transactions have membership_expire_date<transaction_date
memb_exp_test=txn[txn['membership_expire_date']<txn['transaction_date']]
print(memb_exp_test.shape[0])
print(txn.shape[0])
print(memb_exp_test.shape[0]/txn.shape[0])
print(txn['new_id'].nunique())
print(memb_exp_test['new_id'].nunique())

69113
16411725
0.00421119656830711
1079663
57069


In [12]:
#Almost all records with membership_expire_date<transaction_date are auto_renew memberships
print(memb_exp_test[memb_exp_test['is_auto_renew']==1]['is_auto_renew'].sum())
print(txn[txn['is_auto_renew']==1].shape)

68782
(14940882, 12)


In [13]:
#Most of the records with membership_expire_date<transaction_date are cancelled memberships - are these late payments?
#ALL of these are also auto-renew memberships
print(memb_exp_test[memb_exp_test['is_cancel']==1]['is_cancel'].sum())
memb_exp_test[(memb_exp_test['is_cancel']==1)&(memb_exp_test['is_auto_renew']==1)].shape

65833


(65833, 12)

In [14]:
#The rest of the auto_renew memberships
memb_exp_test[(memb_exp_test['is_cancel']==0)&(memb_exp_test['is_auto_renew']==1)].shape

(2949, 12)

In [15]:
#Looks like the rest of the records have lots of instances of membership_expire_date = '1970-01-01'
memb_exp_test[memb_exp_test['is_auto_renew']==0].head(10)
#15 records where membership_expire_date is not is not 1970-01-01 and is not auto_renew
odd_ones_out=memb_exp_test[(memb_exp_test['membership_expire_date']!='1970-01-01') & (memb_exp_test['is_auto_renew']==0)]
#Seems like all of these have plan_list_price of 0
odd_ones_out.head(15)

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,new_id,in_train,odd_transactions,1970
1916276,26,1,0,0,0,2015-12-19,2015-12-18,0,466084,False,True,False
2221659,26,1,0,0,0,2016-04-19,2016-04-17,0,520405,True,True,False
5725996,26,1,0,0,0,2015-10-01,2015-09-30,0,132303,True,True,False
5805657,26,1,0,0,0,2016-08-08,2016-08-06,0,1005530,True,True,False
9530466,26,1,0,0,0,2017-02-23,2017-02-21,0,982520,True,True,False
10277170,26,1,0,0,0,2015-06-15,2015-06-14,0,8236,True,True,False
10315669,26,1,0,0,0,2017-01-07,2017-01-05,0,1011489,True,True,False
10396995,26,0,0,0,0,2015-05-07,2015-05-06,0,797358,True,True,False
10752582,26,1,0,0,0,2015-01-26,2015-01-25,0,77415,True,True,False
11303115,26,1,0,0,0,2016-03-29,2016-03-26,0,838566,False,True,False


In [16]:
#is_cancel=txn[txn['is_cancel']==1]
#print(is_cancel.shape)
#is_cancel['diff_dates']=is_cancel['membership_expire_date']-is_cancel['transaction_date']
#is_cancel['diff_dates']=is_cancel['diff_dates']/ np.timedelta64(1, 'D')
#sns.distplot(is_cancel[is_cancel['diff_dates']>0]['diff_dates'])

In [17]:
#Determine if member had an odd transaction
txn['odd_txn_flag'] = txn['new_id'].isin(memb_exp_test['new_id'])

## Feature Engineering

Each transaction feature is stored in a pandas dataframe:

- **txn_cnts (all members)**: Count number of transactions per member
- **txn_prev_churn (churned members)**: Count number of times members churned, i.e. transaction date - membership expire date from previous transaction is greater than 30 days
- **txn_ar_stop (all members)**: % of transactions that are auto renew and whether a member has stopped auto-renewing in the duration of their membership
- **txn_pmt_change (payment change members)**: Count number of times member changed payment type 
- **txn_cancelled (all members)**: % of transactions that are cancelled and whether a members has cancelled their membership in the duration of their membership
- **txn_cancelled_last (all members)**: If the member has a cancelled transaction as their last transaction

All dfs are to be merged into one df, for transaction attributes per member

Next:
- Look at payment plan days - how often are these 0?
- Look at whether member received a free trial (plan_list_price =0)
- Sum of payment_plan_days
- Transactions matrix

In [76]:
#Prep - sort transactions
txn=txn.sort_values(['new_id', 'transaction_date'])

### Number of Transactions

In [24]:
#Number of transactions
txn_cnts = txn.groupby(['new_id']).size().reset_index(name='txn_cnt')

### Number of Times Churned

In [46]:
#Previous churn: determine if transaction date-membership expiry date from previous row is >30days 
txn['shifted_expiry'] = txn.groupby('new_id')['membership_expire_date'].shift()
txn['memb_gap'] = (txn['transaction_date'] - txn['shifted_expiry'])
txn['prev_churn'] = (txn['memb_gap']/np.timedelta64(1, 'D'))>30
txn.head()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,new_id,in_train,odd_transactions,1970,odd_txn_flag,shifted_expiry,memb_gap,prev_churn
867414,41,30,99,99,1,2016-11-16,2016-12-15,0,0,True,False,False,False,NaT,NaT,False
867416,41,30,99,99,1,2016-12-15,2017-01-15,0,0,True,False,False,False,2016-12-15,0 days,False
867413,41,30,99,99,1,2017-01-15,2017-02-15,0,0,True,False,False,False,2017-01-15,0 days,False
867415,41,30,99,99,1,2017-02-15,2017-03-15,0,0,True,False,False,False,2017-02-15,0 days,False
15369793,39,31,149,149,1,2015-01-31,2015-03-19,0,1,True,False,False,False,NaT,NaT,False


In [37]:
#Aggregate number of times churned
txn_prev_churn = txn[txn['prev_churn'] == True].groupby('new_id')['prev_churn'].count().reset_index(name='prev_churn_cnt')
txn_prev_churn.head()

,new_id,prev_churn_cnt
0,1,1
1,4,1
2,5,1
3,8,2
4,9,1


### Stopped Auto Renew

In [77]:
#Value of 'is_auto_renew' for last transaction
txn_ar_last = txn.groupby(['new_id'])['is_auto_renew'].last().reset_index(name='last_ar')
txn_ar_last['last_ar'].value_counts()

1    917360
0    162303
Name: last_ar, dtype: int64

In [79]:
#Changed auto renew:
txn_ar_change = txn.groupby(['new_id'])['is_auto_renew'].mean().reset_index(name='mean_ar')
txn_ar_stop = txn_ar_change.merge(txn_ar_last, on='new_id', how='inner')
txn_ar_stop['stopped_ar']=(txn_ar_stop['mean_ar'] < 1) & (txn_ar_stop['mean_ar'] > 0) & (txn_ar_stop['last_ar'] == 0)
txn_ar_stop[txn_ar_stop['stopped_ar']==True].head()

,new_id,mean_ar,last_ar,stopped_ar
9,9,0.571429,0,True
80,80,0.857143,0,True
135,135,0.714286,0,True
243,243,0.875000,0,True
265,265,0.875000,0,True


In [80]:
txn_ar_stop['stopped_ar'].value_counts()

False    1062259
True       17404
Name: stopped_ar, dtype: int64

### Changed Payment Method

In [52]:
#Changed payment method:determine if transaction date-membership expiry date from previous row is >30days 
txn['shifted_pmt'] = txn.groupby('new_id')['payment_method_id'].shift()
txn['pmt_change'] = (txn['payment_method_id'] != txn['shifted_pmt']) & (txn['shifted_pmt'].isnull() != True)
print(txn.head())
txn['pmt_change'].value_counts()

          payment_method_id  payment_plan_days  plan_list_price  \
867414                   41                 30               99   
867416                   41                 30               99   
867413                   41                 30               99   
867415                   41                 30               99   
15369793                 39                 31              149   

          actual_amount_paid  is_auto_renew transaction_date  \
867414                    99              1       2016-11-16   
867416                    99              1       2016-12-15   
867413                    99              1       2017-01-15   
867415                    99              1       2017-02-15   
15369793                 149              1       2015-01-31   

         membership_expire_date  is_cancel  new_id in_train odd_transactions  \
867414               2016-12-15          0       0     True            False   
867416               2017-01-15          0       0  

False    16138228
True       273497
Name: pmt_change, dtype: int64

In [60]:
#Aggregate number of times changed payment method
txn_pmt_change = txn[txn['pmt_change'] == True].groupby('new_id')['pmt_change'].count().reset_index(name='pmt_change_cnt')
txn_pmt_change.head()

,new_id,pmt_change_cnt
0,7,2
1,8,2
2,9,2
3,21,3
4,36,1


### Cancelled Membership

In [62]:
#Cancelled membership:
txn_cancelled = txn.groupby(['new_id'])['is_cancel'].mean().reset_index(name='mean_cancel')
txn_cancelled['cancelled']=(txn_cancelled['mean_cancel']<1)&(txn_cancelled['mean_cancel']>0)
txn_cancelled[txn_cancelled['cancelled']==True].head()

,new_id,mean_cancel,cancelled
9,9,0.142857,True
17,17,0.037037,True
21,21,0.071429,True
28,28,0.050000,True
35,35,0.133333,True


In [70]:
txn_cancelled['cancelled'].value_counts()

False    846452
True     233211
Name: cancelled, dtype: int64

In [69]:
#Cancelled membership in last transaction:
txn_cancelled_last = txn.groupby(['new_id'])['is_cancel'].last().reset_index(name='last_cancel')
txn_cancelled_last['last_cancel'].value_counts()

0    1047277
1      32386
Name: last_cancel, dtype: int64